In [24]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dense
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dense
import keras_nlp

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import copy
warnings.simplefilter(action='ignore')

1. if movie 2 is watched after movie 1 (doesn't need to be right after), movie 2's rating is N(1, 1). otherwise, movie 2's rating is N(5,1).
2. if movie 4 is watched right after movie 3, movie 4's rating N(1,1). if movie 3 is watched right after movie 4, movie 3's rating is N(1,1).
3. if movie 5 is watched last, its rating is N(5, 1)

In [5]:
# Parameters
N = 10000  # Number of users
M = 5  # Number of movies
sigma = 1  # Standard deviation for ratings

ratings = []  # Ratings for all users
orders = []  # Viewing orders for all users

np.random.seed(42)

for i in range(N):
    order = np.random.permutation(M) + 1 
    orders.append(order)
    
    user_ratings = []
    
    for j, movie in enumerate(order):
        if movie == 2:
            # Movie 2 after Movie 1
            if 1 in order[:j]:
                rating = np.random.normal(1, sigma)
            else:
                rating = np.random.normal(5, sigma)
        
        elif movie == 4:
            # Movie 4 immediately after Movie 3
            if j > 0 and order[j - 1] == 3:
                rating = np.random.normal(1, sigma)
            else:
                rating = np.random.normal(3, sigma)
        
        elif movie == 3:
            # Movie 3 immediately after Movie 4
            if j > 0 and order[j - 1] == 4:
                rating = np.random.normal(1, sigma)
            else:
                rating = np.random.normal(3, sigma)
        
        elif movie == 5:
            # Movie 5 watched last
            if j == M - 1:
                rating = np.random.normal(5, sigma)
            else:
                rating = np.random.normal(3, sigma)
        
        else:
            # Default rating for other cases
            rating = np.random.normal(3, sigma)
        
        user_ratings.append(rating)
    
    ratings.append(user_ratings)

ratings = np.array(ratings)
orders = np.array(orders)

# Example output
print("Example user order:", orders[0])
print("Ratings for the user:", ratings[0])

Example user order: [2 5 3 1 4]
Ratings for the user: [5.47386083 4.36845012 2.08317316 2.87585282 0.98903711]


In [13]:
data = []

# Populate the data list
for user_id in range(N):
    for timestamp, movie_id in enumerate(orders[user_id], start=1):
        data.append({
            "user_id": user_id + 1,
            "movie_id": movie_id,
            "rating": ratings[user_id][timestamp - 1],
            "timestamp": timestamp
        })

data = pd.DataFrame(data)

In [15]:
data

,user_id,movie_id,rating,timestamp
0,1,2,5.473861,1
1,1,5,4.368450,2
2,1,3,2.083173,3
3,1,1,2.875853,4
4,1,4,0.989037,5
...,...,...,...,...
49995,10000,4,2.627712,1
49996,10000,1,3.590464,2
49997,10000,2,0.972798,3
49998,10000,3,4.267548,4


In [17]:
data_train = data[data['user_id'] <= 7500].reset_index(drop = True)
data_test = data[data['user_id'] >= 7501].reset_index(drop = True)

In [20]:
def preprocess_data(df):
    
    center = []
    movie = []
    rating = []
    length = []
    target = []
    
    unique_user_ids = df['user_id'].unique()
    
    for user_id in unique_user_ids:

        temp = df[df['user_id'] == user_id]
        movie_list, rating_list = list(temp['movie_id']), list(temp['rating'])
        center += movie_list
        target += rating_list
        
        for idx in range(temp.shape[0]):
            movie.append(movie_list[:idx] + [0] * (temp.shape[0] - idx))
            rating.append(rating_list[:idx] + [0] * (temp.shape[0] - idx))
            length.append(idx if idx > 0 else 1)
            
    return center, movie, rating, length, target

In [21]:
center_train, movie_train, rating_train, length_train, target_train \
    = preprocess_data(data_train)

center_test, movie_test, rating_test, length_test, target_test \
    = preprocess_data(data_test)

In [26]:
center_train, center_val, movie_train, movie_val, rating_train, rating_val, \
length_train, length_val, target_train, target_val = train_test_split(
    center_train, movie_train, rating_train, length_train, target_train, 
    test_size=0.25, random_state=42
)

In [27]:
max_ctx = max(length_train + length_test + length_val)

In [28]:
movie_train = [movie + [0] * (max_ctx - len(movie)) if len(movie) < max_ctx 
               else movie[:max_ctx] for movie in movie_train]

movie_val = [movie + [0] * (max_ctx - len(movie)) if len(movie) < max_ctx 
               else movie[:max_ctx] for movie in movie_val]

movie_test = [movie + [0] * (max_ctx - len(movie)) if len(movie) < max_ctx 
               else movie[:max_ctx] for movie in movie_test]

rating_train = [rating + [0] * (max_ctx - len(rating)) if len(rating) < max_ctx 
               else rating[:max_ctx] for rating in rating_train]

rating_val = [rating + [0] * (max_ctx - len(rating)) if len(rating) < max_ctx 
               else rating[:max_ctx] for rating in rating_val]

rating_test = [rating + [0] * (max_ctx - len(rating)) if len(rating) < max_ctx 
               else rating[:max_ctx] for rating in rating_test]

In [29]:
center_train_data = np.array(center_train)  # Center movie
context_train_data = np.array(movie_train)  # Context movies
rating_train_data = np.array(rating_train)  # Context ratings
length_train_data = np.array(length_train)  # Length of the context
target_train_data = np.array(target_train)  # Target rating

center_val_data = np.array(center_val)  # Center movie
context_val_data = np.array(movie_val)  # Context movies
rating_val_data = np.array(rating_val)  # Context ratings
length_val_data = np.array(length_val)  # Length of the context
target_val_data = np.array(target_val)  # Target rating

center_test_data = np.array(center_test)  # Center movie
context_test_data = np.array(movie_test)  # Context movies
rating_test_data = np.array(rating_test)  # Context ratings
length_test_data = np.array(length_test)  # Length of the context
target_test_data = np.array(target_test)  # Target rating

In [33]:
center_test_data

array([4, 2, 5, ..., 2, 3, 5])

In [34]:
context_test_data 

array([[0, 0, 0, 0],
       [4, 0, 0, 0],
       [4, 2, 0, 0],
       ...,
       [4, 1, 0, 0],
       [4, 1, 2, 0],
       [4, 1, 2, 3]])

In [35]:
rating_test_data

array([[0.        , 0.        , 0.        , 0.        ],
       [3.86321173, 0.        , 0.        , 0.        ],
       [3.86321173, 3.33368923, 0.        , 0.        ],
       ...,
       [2.62771237, 3.59046371, 0.        , 0.        ],
       [2.62771237, 3.59046371, 0.97279766, 0.        ],
       [2.62771237, 3.59046371, 0.97279766, 4.2675476 ]])

In [36]:
length_test_data

array([1, 1, 2, ..., 2, 3, 4])

In [37]:
target_test_data

array([3.86321173, 3.33368923, 3.24878386, ..., 0.97279766, 4.2675476 ,
       4.08073598])

In [39]:
embedding_size = 32
input_dim = data['movie_id'].nunique() + 1

center_input = layers.Input(shape=(1,), dtype=tf.int32, name="center_input")  
context_input = layers.Input(shape=(max_ctx,), dtype=tf.int32, name="context_input")  
rating_input = layers.Input(shape=(max_ctx,), dtype=tf.float32, name="rating_input")  
length_input = layers.Input(shape=(1,), dtype=tf.float32, name="length_input") 

movie_embedding = layers.Embedding(input_dim=input_dim, output_dim=embedding_size)(context_input)
center_embedding = layers.Embedding(input_dim=input_dim, output_dim=embedding_size)(center_input)

rating_input = layers.Reshape((max_ctx, 1))(rating_input)
weighted_embeddings = layers.Multiply()([movie_embedding, rating_input])

sum_embeddings = layers.Lambda(lambda x: tf.reduce_sum(x, axis=1), name="sum_embeddings")(weighted_embeddings)

average_embeddings = layers.Lambda(lambda inputs: inputs[0] / inputs[1], name="average_embeddings")(
    [sum_embeddings, length_input]
)

center_embedding_flat = layers.Flatten()(center_embedding)
lambda_output = layers.Dot(axes=-1, name="dot_product")([average_embeddings, center_embedding_flat])
lambda_output_exp = layers.Activation('linear', name="lambda_output_exp")(lambda_output)

model = Model(inputs=[center_input, context_input, rating_input, length_input], outputs=lambda_output_exp)

model.compile(optimizer=Adam(learning_rate = 1e-4), loss='mean_squared_error')

early_stopping = EarlyStopping(
    monitor='val_loss',    
    patience=10,  
    restore_best_weights=True 
)

# Fit the model with validation data and early stopping
history = model.fit(
    [center_train_data, context_train_data, rating_train_data, length_train_data], 
    target_train_data,        
    validation_data=([center_val_data, context_val_data, rating_val_data, length_val_data], target_val_data),  
    epochs=1000,                 
    batch_size=256,            
    callbacks=[early_stopping] 
)

Epoch 1/1000
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 10.7083 - val_loss: 9.3646
Epoch 2/1000
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8.5558 - val_loss: 5.7734
Epoch 3/1000
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.3288 - val_loss: 5.0657
Epoch 4/1000
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.9328 - val_loss: 4.9890
Epoch 5/1000
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.7982 - val_loss: 4.9457
Epoch 6/1000
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.7740 - val_loss: 4.9059
Epoch 7/1000
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.7359 - val_loss: 4.8645
Epoch 8/1000
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.5879 - val_loss: 4.8235
Epoch 9/1000
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.6836 - val_loss: 4.7857
Epoch 10/1000
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.5903 - val_loss: 4.7537
Epoch 11/1000
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.5364 - val_loss: 4.7284
Epoch 12/1000
110/110 ━━━━━━━

In [40]:
test_loss = model.evaluate([center_test_data, context_test_data, rating_test_data, length_test_data], target_test_data)
print(f"Test Loss: {test_loss}")

391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step - loss: 4.4728
Test Loss: 4.518759250640869
